In [ ]:
!pip install pandas scikit-learn tqdm ipywidgets spacy>pip.log

!python -m spacy download ru_core_news_lg>spacy.log

2023-05-22 20:30:03.183519: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-22 20:30:04.514931: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
import numpy as np
import pandas as pd
import spacy
import joblib
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

tqdm.pandas()

In [ ]:
from google.colab import drive
drive.mount('/content/drive', True)

DATA_DIR = '/content/drive/MyDrive/aeroclub/data'
SUBM_DIR = '/content/drive/MyDrive/aeroclub/submission'

!ls {DATA_DIR}

Mounted at /content/drive
groundtruth_data.csv		   supervised_data_preprocessed.csv
groundtruth_data_preprocessed.csv  test_data_cleared.csv
labeled_data.csv		   test_data_preprocessed.csv
labeled_data_preprocessed.csv	   traind_data.csv
manual_check_data.csv		   traind_data_preprocessed.csv
supervised_data.csv


In [ ]:
disable = ['tok2vec', 'tagger', 'morphologizer', 'parser', 'senter', 'attribute_ruler', 'ner']

nlp = spacy.load("ru_core_news_lg", disable=disable)

In [ ]:
def preprocess_body(text):
    text = text.lower()
    
    # убираю часть после re: когда цитируется сообщение, на которое составляется ответ
    text = text.split('re:', 1)[0]
    text = text.split(' re ', 1)[0]
    
    # убираю подписи
    text = text.split('с уважением', 1)[0]
    text = text.split('this e-mail and its attachments', 1)[0]
    text = text.split('this e-mail and all attachments', 1)[0]
    text = text.split('this message and its attachments', 1)[0]
    text = text.split('this message and all attachments', 1)[0]
    text = text.split('information contained in this email', 1)[0]
    
    # ...
    
    return text


def lemmatize_text(text):
    doc = nlp(text)
    return ' '.join([t.lemma_ for t in doc])


def preprocess_df(df):
    title = df['title'].str.lower().fillna('')

    print('Extracting payload...')
    payload = df['text'].fillna('').progress_apply(preprocess_body).rename('payload')

    print('Lemmatizing text...')
    title_lemmas = title.progress_apply(lemmatize_text).rename('title_lemmas')
    payload_lemmas = payload.progress_apply(lemmatize_text).rename('payload_lemmas')
    
    title_payload = title.rename('title_payload')
    title_payload += ' payload '
    title_payload += payload

    title_payload_lemmas = title_lemmas.rename('title_payload_lemmas')
    title_payload_lemmas += ' payload '
    title_payload_lemmas += payload_lemmas

    result = pd.concat((df, payload, title_payload, title_lemmas, payload_lemmas, title_payload_lemmas), axis=1)

    return result


### Raw data

In [ ]:
df_raw = pd.read_csv(f'{DATA_DIR}/traind_data.csv')
df_raw

,id,title,text
0,0,[ Aeroclub NAME NAME NAME PASSPORT/409285. Соо...,Здравствуйте! NAME Командировка PASSPORT/4092...
1,1,[ Aeroclub NAME NAME NAME PASSPORT/409285. Соо...,Здравствуйте! NAME Командировка PASSPORT/4092...
2,2,[ Aeroclub NAME ] Командировка PASSPORT/339029...,Здравствуйте! NAME Командировка PASSPORT/3390...
3,3,[ Aeroclub NAME ] Командировка PASSPORT/357530...,Здравствуйте! NAME Командировка PASSPORT/3575...
4,4,[ Aeroclub NAME NAME NAME PASSPORT Изменение у...,Здравствуйте! Сяинова NAME NAME NAME NAME NAM...
...,...,...,...
14029,14029,Прошу оформить билеты по бронированию 1A / WXN...,Прошу оформить билеты по бронированию 1A / WXN...
14030,14030,Прошу оформить билеты по бронированию 1G/9VGPP...,Прошу оформить билеты по бронированию 1G/9VGPP...
14031,14031,RE: M-476284 || Запрос бронирования NAME NAME ...,"Коллеги, Просим прислать бронирования для врач..."
14032,14032,RE: M-476284 || Запрос бронирования NAME NAME ...,"Уважаемые коллеги, 3го числа экстрасит не нуже..."


In [ ]:
df_raw_preprocessed = preprocess_df(df_raw)
df_raw_preprocessed.to_csv(f'{DATA_DIR}/traind_data_preprocessed.csv', index=False)

df_raw_preprocessed.head(2)

Extracting payload...


100%|██████████| 14034/14034 [00:00<00:00, 25427.06it/s]


Lemmatizing text...


100%|██████████| 14034/14034 [02:29<00:00, 93.69it/s]


,id,title,text,payload,title_payload,title_lemmas,payload_lemmas,title_payload_lemmas
0,0,[ Aeroclub NAME NAME NAME PASSPORT/409285. Соо...,Здравствуйте! NAME Командировка PASSPORT/4092...,здравствуйте! name командировка passport/4092...,[ aeroclub name name name passport/409285. соо...,[ aeroclub name name name passport/409285 . со...,здравствуйте ! name командировка passport/40...,[ aeroclub name name name passport/409285 . со...
1,1,[ Aeroclub NAME NAME NAME PASSPORT/409285. Соо...,Здравствуйте! NAME Командировка PASSPORT/4092...,здравствуйте! name командировка passport/4092...,[ aeroclub name name name passport/409285. соо...,[ aeroclub name name name passport/409285 . со...,здравствуйте ! name командировка passport/40...,[ aeroclub name name name passport/409285 . со...


### Manual labeled data (part of raw data) labeled by expert teammate


In [ ]:
df_supervised = pd.read_csv(f'{DATA_DIR}/supervised_data.csv')
df_supervised

,id,target,title,text
0,9,0,Re: RE: NAME NAME,"PASSPORT С уважением/ NAME regards, NAME NAME ..."
1,11,1,RE: Подтверждение бронирования: 09.10. NAME NA...,Добрый день! NAME Подтверждаю! ---------------...
2,12,1,RE: Подтверждение бронирования: 09.10. Vyborny...,здравствуйте. NAME подтверждаю. --------------...
3,14,1,"Re: Варианты проживания ( Ставрополь, Россия, ...",Добрый день! Мне подойдёт второй вариант отель...
4,15,1,NAME,"NAME, Бронь подтверждаю. С уважением, NAME NAM..."
...,...,...,...,...
710,5039,1,Re: RE: RE: Краснодар,"выписываем С уважением/ NAME regards, NAME NAM..."
711,5040,1,"RE: NAME NAME, 04.11",Добрый день! Пункт 2 Спасибо! From: Aeroclub N...
712,5042,0,RE: Камеха / Цюрих,NAME NAME NAME предложить NAME NAME NAME NAME ...
713,5043,1,FW: NAME номер приказа 6262 на командировку NA...,"Коллеги, добрый день! Прошу забронировать гост..."


In [ ]:
df_supervised_preprocessed = preprocess_df(df_supervised)
df_supervised_preprocessed.to_csv(f'{DATA_DIR}/supervised_data_preprocessed.csv', index=False)

df_supervised_preprocessed.head(2)

Extracting payload...


100%|██████████| 715/715 [00:00<00:00, 17976.06it/s]


Lemmatizing text...


100%|██████████| 715/715 [00:05<00:00, 128.02it/s]


,id,target,title,text,payload,title_payload,title_lemmas,payload_lemmas,title_payload_lemmas
0,9,0,Re: RE: NAME NAME,"PASSPORT С уважением/ NAME regards, NAME NAME ...",passport,re: re: name name payload passport,re : re : name name,passport,re : re : name name payload passport
1,11,1,RE: Подтверждение бронирования: 09.10. NAME NA...,Добрый день! NAME Подтверждаю! ---------------...,добрый день! name подтверждаю! ---------------...,re: подтверждение бронирования: 09.10. name na...,re : подтверждение бронирование : 09.10 . name...,добрый день ! name подтверждать ! ------------...,re : подтверждение бронирование : 09.10 . name...


### Manual labeled data (part of raw data) labeled by different teammates


In [ ]:
df_labeled = pd.read_csv(f'{DATA_DIR}/labeled_data.csv')
df_labeled

,id,target,title,text
0,9,0,Re: RE: NAME NAME,"PASSPORT С уважением/ NAME regards, NAME NAME ..."
1,11,1,RE: Подтверждение бронирования: 09.10. NAME NA...,Добрый день! NAME Подтверждаю! ---------------...
2,12,1,RE: Подтверждение бронирования: 09.10. Vyborny...,здравствуйте. NAME подтверждаю. --------------...
3,14,1,"Re: Варианты проживания ( Ставрополь, Россия, ...",Добрый день! Мне подойдёт второй вариант отель...
4,15,1,NAME,"NAME, Бронь подтверждаю. С уважением, NAME NAM..."
...,...,...,...,...
4141,642,1,RE: Венеция.,"NAME, добрый день NAME Пришлите, пожалуйста, н..."
4142,684,1,Re: [ Aeroclub NAME NAME NAME PASSPORT/409554....,"Коллеги, есть ли новости по данному запросу? N..."
4143,728,1,Подтверждение бронирования № PASSPORT,Подтверждение бронирования № PASSPORT < NAME >...
4144,748,1,Подтверждение бронирования № PASSPORT,Подтверждение бронирования № PASSPORT < NAME >...


In [ ]:
df_dupl = pd.concat((df_labeled, df_supervised), axis=0) \
    .loc[:, 'id'] \
    .value_counts() \
    .rename('count') \
    .rename_axis('id') \
    .reset_index() \
    .loc[lambda df: df['count'] > 1, ['id']]

df_dupl

,id
0,9
1,2182
2,2161
3,2163
4,2164
...,...
710,1048
711,1051
712,1052
713,1053


In [ ]:
df_labeled = df_labeled[~df_labeled['id'].isin(df_dupl['id'].unique())].copy()

pd.concat((df_labeled, df_supervised), axis=0) \
    .loc[:, 'id'] \
    .value_counts() \
    .rename('count') \
    .rename_axis('id') \
    .reset_index() \
    .loc[lambda df: df['count'] > 1, ['id']]

,id


In [ ]:
df_labeled_preprocessed = preprocess_df(df_labeled)
df_labeled_preprocessed.to_csv(f'{DATA_DIR}/labeled_data_preprocessed.csv', index=False)

df_labeled_preprocessed.head(2)

Extracting payload...


100%|██████████| 3431/3431 [00:00<00:00, 15721.55it/s]


Lemmatizing text...


100%|██████████| 3431/3431 [00:40<00:00, 85.61it/s]


,id,target,title,text,payload,title_payload,title_lemmas,payload_lemmas,title_payload_lemmas
715,27,0,[ Aeroclub NAME NAME NAME PASSPORT. Выписаны б...,"Здравствуйте! Труб NAME NAME ( "" КИВИ "" ) офо...","здравствуйте! труб name name ( "" киви "" ) офо...",[ aeroclub name name name passport. выписаны б...,[ aeroclub name name name passport . выписать ...,"здравствуйте ! труба name name ( "" киви "" ) ...",[ aeroclub name name name passport . выписать ...
716,139,0,[ Aeroclub NAME NAME NAME PASSPORT. Оформление...,"Good day! Игнатьева NAME NAME ( "" КИВИ БАНК ""...","good day! игнатьева name name ( "" киви банк ""...",[ aeroclub name name name passport. оформление...,[ aeroclub name name name passport . оформлени...,"good day ! игнатьев name name ( "" киви банк ...",[ aeroclub name name name passport . оформлени...


### Ground truth data

In [ ]:
df_truth = pd.read_csv(f'{DATA_DIR}/groundtruth_data.csv')
df_truth = df_truth[['id', 'label', 'title', 'text']].copy()
df_truth

,id,target,title,text
0,11500,1,"Re: Ваучер к заказу 4605077: Адажио, Санкт - П...","NAME, добрый день! Прошу Вас включить ранний з..."
1,11501,1,RE: обмен билета для NAME NAME с 9 на 11 декабря,"NAME, добрый день, Пожалуйста, переписывайте с..."
2,11502,1,RE: а / бронь 6 декабря Самара - Москва - Сама...,"NAME, да, выписываем. NAME Спасибо! NAME With ..."
3,11503,1,RE: * * RE: билеты на Майорку * * * External S...,"Скажите, а такой вариант, но дата обратно – 18..."
4,11504,0,Re: Ранний заезд.,"NAME, рз в 08:00 стоит 1525 рублей. С уважение..."
...,...,...,...,...
495,11995,1,Re: NAME NAME NAME,"Хорошо, берём NAME NAME NAME для iOS < NAME > ..."
496,11996,1,RE: [ EXTERNAL ] RE: NAME номер приказа 6289 н...,"NAME, тогда жду до конца дня инфо от вас, как ..."
497,11997,0,"RE: Ваучер к заказу 4587374: NAME NAME NAME, N...","Добрый день, NAME Я сразу же пошел поскандалил..."
498,11998,1,"RE: Электронный билет: # 421-PASSPORT, NAME NA...","NAME, добрый день. NAME. Пришлите, пожалуйста,..."


In [ ]:
df_truth_preprocessed = preprocess_df(df_truth)
df_truth_preprocessed.to_csv(f'{DATA_DIR}/groundtruth_data_preprocessed.csv', index=False)

df_truth_preprocessed.head(2)

Extracting payload...


100%|██████████| 500/500 [00:00<00:00, 10713.53it/s]


Lemmatizing text...


100%|██████████| 500/500 [00:09<00:00, 54.45it/s]


,id,target,title,text,payload,title_payload,title_lemmas,payload_lemmas,title_payload_lemmas
0,11500,1,"Re: Ваучер к заказу 4605077: Адажио, Санкт - П...","NAME, добрый день! Прошу Вас включить ранний з...","name, добрый день! прошу вас включить ранний з...","re: ваучер к заказу 4605077: адажио, санкт - п...","re : ваучер к заказ 4605077 : адажио , санкт -...","name , добрый день ! просить вы включить ранни...","re : ваучер к заказ 4605077 : адажио , санкт -..."
1,11501,1,RE: обмен билета для NAME NAME с 9 на 11 декабря,"NAME, добрый день, Пожалуйста, переписывайте с...","name, добрый день, пожалуйста, переписывайте с...",re: обмен билета для name name с 9 на 11 декаб...,re : обмен билет для name name с 9 на 11 декабрь,"name , добрый день , пожалуйста , переписывать...",re : обмен билет для name name с 9 на 11 декаб...


In [ ]:
df_validate = pd.read_csv(f'{DATA_DIR}/manual_check_data.csv')
df_validate = df_validate[['id', 'label', 'title', 'text']].copy()
df_validate

,id,target,title,text
0,11000,0,"RE: 555-PASSPORT, NAME NAME","NAME NAME. Для дополнительных консультаций, п..."
1,11001,0,Анапа,# # # write your NAME NAME NAME line # # #:: 5...
2,11002,1,"RE: заявка гост. NAME, Анапа","NAME, добрый вечер! Не получила ваучер, Пионер..."
3,11003,1,"FW: заявка гост. NAME, Анапа",Анна!пионерский NAME NAME Спасибо NAME С уваже...
4,11004,0,RE: билеты для NAME,"Ок С уважением, NAME NAME Менеджер по организа..."
...,...,...,...,...
995,11995,1,Re: NAME NAME NAME,"Хорошо, берём NAME NAME NAME для iOS < NAME > ..."
996,11996,1,RE: [ EXTERNAL ] RE: NAME номер приказа 6289 н...,"NAME, тогда жду до конца дня инфо от вас, как ..."
997,11997,0,"RE: Ваучер к заказу 4587374: NAME NAME NAME, N...","Добрый день, NAME Я сразу же пошел поскандалил..."
998,11998,1,"RE: Электронный билет: # 421-PASSPORT, NAME NA...","NAME, добрый день. NAME. Пришлите, пожалуйста,..."


In [ ]:
df_validate_preprocessed = preprocess_df(df_truth)
df_validate_preprocessed.to_csv(f'{DATA_DIR}/validate_data_preprocessed.csv', index=False)

df_validate_preprocessed.head(2)

### Submission data

In [ ]:
df_submission = pd.read_csv(f'{DATA_DIR}/test_data_cleared.csv')
df_submission

,id,title,text,label
0,11000,"RE: 555-PASSPORT, NAME NAME","NAME NAME. Для дополнительных консультаций, п...",-1
1,11001,Анапа,# # # write your NAME NAME NAME line # # #:: 5...,-1
2,11002,"RE: заявка гост. NAME, Анапа","NAME, добрый вечер! Не получила ваучер, Пионер...",-1
3,11003,"FW: заявка гост. NAME, Анапа",Анна!пионерский NAME NAME Спасибо NAME С уваже...,-1
4,11004,RE: билеты для NAME,"Ок С уважением, NAME NAME Менеджер по организа...",-1
...,...,...,...,...
995,11995,Re: NAME NAME NAME,"Хорошо, берём NAME NAME NAME для iOS < NAME > ...",-1
996,11996,RE: [ EXTERNAL ] RE: NAME номер приказа 6289 н...,"NAME, тогда жду до конца дня инфо от вас, как ...",-1
997,11997,"RE: Ваучер к заказу 4587374: NAME NAME NAME, N...","Добрый день, NAME Я сразу же пошел поскандалил...",-1
998,11998,"RE: Электронный билет: # 421-PASSPORT, NAME NA...","NAME, добрый день. NAME. Пришлите, пожалуйста,...",-1


In [ ]:
df_submission_preprocessed = preprocess_df(df_submission)
df_submission_preprocessed.to_csv(f'{DATA_DIR}/test_data_preprocessed.csv', index=False)

df_submission_preprocessed.head(2)

Extracting payload...


100%|██████████| 1000/1000 [00:00<00:00, 4050.66it/s]


Lemmatizing text...


100%|██████████| 1000/1000 [00:12<00:00, 82.89it/s]


,id,title,text,label,payload,title_payload,title_lemmas,payload_lemmas,title_payload_lemmas
0,11000,"RE: 555-PASSPORT, NAME NAME","NAME NAME. Для дополнительных консультаций, п...",-1,"name name. для дополнительных консультаций, п...","re: 555-passport, name name payload name name...","re : 555-passport , name name",name name . для дополнительный консультация ...,"re : 555-passport , name name payload name n..."
1,11001,Анапа,# # # write your NAME NAME NAME line # # #:: 5...,-1,# # # write your name name name line # # #:: 5...,анапа payload # # # write your name name name ...,анапа,# # # write your name name name line # # # : :...,анапа payload # # # write your name name name ...


In [ ]:
!ls {DATA_DIR}

groundtruth_data.csv		   supervised_data_preprocessed.csv
groundtruth_data_preprocessed.csv  test_data_cleared.csv
labeled_data.csv		   test_data_preprocessed.csv
labeled_data_preprocessed.csv	   traind_data.csv
supervised_data.csv		   traind_data_preprocessed.csv
